In [1]:
# Loading raw text dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-14 19:00:31--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.69MB/s    in 0.6s    

2025-10-14 19:00:32 (1.69 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
with open("input.txt","r") as f:
    data = f.read()

In [2]:
print("total characters in the data:",len(data))
print("total unique characters in the data:",len(list(set(data))))

total characters in the data: 1115394
total unique characters in the data: 65


In [3]:
print("Tiny sample of the data")
print(data[:500])

Tiny sample of the data
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
## Store unique char and vocab size
unique_char = sorted(list(set(data)))
vocab_size = len(unique_char)
print("Unique characters in the data")
print(''.join(unique_char))

Unique characters in the data

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
## simple text to token generation
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(unique_char) }
itos = { i:ch for i,ch in enumerate(unique_char) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

ecd = encode("gpt from scratch")
print(ecd)
print(decode(ecd))

[45, 54, 58, 1, 44, 56, 53, 51, 1, 57, 41, 56, 39, 58, 41, 46]
gpt from scratch


In [6]:
# Encode the whole data
import torch
encoded_data = torch.tensor(encode(data), dtype = torch.long)
print("Encoded data:\n",encoded_data[:100])

Encoded data:
 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
# Train test split
n = int(0.9*len(encoded_data))
train_data = encoded_data[:n]
val_data = encoded_data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
# inputs and target
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [10]:
torch.manual_seed(21)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[43, 39, 56, 58, 57,  2,  0,  0],
        [ 1, 45, 53, 53, 42,  1, 58, 53],
        [44,  1, 39, 52, 53, 58, 46, 43],
        [58, 61, 47, 52, 52,  5, 42,  1]])
targets:
torch.Size([4, 8])
tensor([[39, 56, 58, 57,  2,  0,  0, 32],
        [45, 53, 53, 42,  1, 58, 53,  1],
        [ 1, 39, 52, 53, 58, 46, 43, 56],
        [61, 47, 52, 52,  5, 42,  1, 50]])
----
when input is [43] the target is 39
when input is [43, 39] the target is 56
when input is [43, 39, 56] the target is 58
when input is [43, 39, 56, 58] the target is 57
when input is [43, 39, 56, 58, 57] the target is 2
when input is [43, 39, 56, 58, 57, 2] the target is 0
when input is [43, 39, 56, 58, 57, 2, 0] the target is 0
when input is [43, 39, 56, 58, 57, 2, 0, 0] the target is 32
when input is [1] the target is 45
when input is [1, 45] the target is 53
when input is [1, 45, 53] the target is 53
when input is [1, 45, 53, 53] the target is 42
when input is [1, 45, 53, 53, 42] the target is

In [11]:
# Simple bigram language model
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.7627, grad_fn=<NllLossBackward0>)

DjbJGMfeLPQXfSh V,u-yFS HMcgxEOHD-EBx
MAWG&$nUfNye.QzWRYOzfyeqQWp?XLk.N,xExh: !
DAW- bRQOFTirDnhzA&s


In [24]:
nn.Embedding(vocab_size, vocab_size)(xb)

tensor([[[ 0.4091,  0.5536, -0.6355,  ...,  0.2315,  0.7595, -0.1151],
         [-0.8714,  0.3461,  1.5941,  ..., -1.5589,  0.5345,  2.0063],
         [ 0.9967,  0.2983, -0.2620,  ...,  0.1545, -0.4561, -1.4559],
         ...,
         [ 0.3117, -0.2086, -2.4545,  ..., -0.4868,  2.5360,  0.2891],
         [-0.4060,  1.3679, -0.6643,  ...,  0.9013,  0.3211,  0.4461],
         [-0.4060,  1.3679, -0.6643,  ...,  0.9013,  0.3211,  0.4461]],

        [[ 1.6119, -0.6937,  0.3052,  ...,  0.1244, -0.4948,  1.2786],
         [ 0.9485,  1.9939, -1.1560,  ...,  1.8434, -1.4623, -1.0992],
         [-0.3712, -0.5019, -0.3782,  ..., -2.1009, -0.0803,  0.4078],
         ...,
         [ 1.6119, -0.6937,  0.3052,  ...,  0.1244, -0.4948,  1.2786],
         [-0.7149,  1.6099,  1.5141,  ...,  3.7616,  0.7390,  1.1754],
         [-0.3712, -0.5019, -0.3782,  ..., -2.1009, -0.0803,  0.4078]],

        [[ 0.3379, -0.5611, -0.1635,  ...,  0.1496,  0.9633,  0.1509],
         [ 1.6119, -0.6937,  0.3052,  ...,  0

In [25]:
yb.view(32)

tensor([39, 56, 58, 57,  2,  0,  0, 32, 45, 53, 53, 42,  1, 58, 53,  1,  1, 39,
        52, 53, 58, 46, 43, 56, 61, 47, 52, 52,  5, 42,  1, 50])

In [26]:
# optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [27]:
batch_size = 32
for steps in range(100000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.397651195526123


In [28]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


EShise
Co, NTILAcoutit; hat l d be seimyoyond s pon jathaswele.
S:




I'se bomode.
NT:
Hind! od hir tiloubuchered RESAnt hthatoo e my don, ain he scorinond hend mbethatik my; h onibld h!
NGRIELO, aco angofe se.
OLERLAlf
Irte the acough,
Herdrr sthe w'lina RERDooplish ayourdids auringhit bame e, an.
Wht.
Arvin bsive w ors bonas wain fo,' mises ck ard musoureallo RD ay painis us wn w:
Wham! u pave avealily.


May bardin Wevyengith ayoun cl t we finoustorans thendnte wheentonanwavery athichitanoor


In [30]:
# matric tricks
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x[0]

tensor([[ 0.1839, -0.0461],
        [-2.3070,  0.1532],
        [-0.6190,  0.8345],
        [-1.3548,  0.8263],
        [-0.2921, -0.6292],
        [ 0.5261, -1.2520],
        [-1.2366,  1.0383],
        [-0.1763, -0.4746]])

In [31]:
ones = torch.ones(T,T)
ones

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [34]:
x[0]

tensor([[ 0.1839, -0.0461],
        [-2.3070,  0.1532],
        [-0.6190,  0.8345],
        [-1.3548,  0.8263],
        [-0.2921, -0.6292],
        [ 0.5261, -1.2520],
        [-1.2366,  1.0383],
        [-0.1763, -0.4746]])

In [33]:
ones @ x[0]

tensor([[-5.2759,  0.4504],
        [-5.2759,  0.4504],
        [-5.2759,  0.4504],
        [-5.2759,  0.4504],
        [-5.2759,  0.4504],
        [-5.2759,  0.4504],
        [-5.2759,  0.4504],
        [-5.2759,  0.4504]])

In [35]:
ones = torch.tril(torch.ones(T,T))
ones

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [36]:
x[0]

tensor([[ 0.1839, -0.0461],
        [-2.3070,  0.1532],
        [-0.6190,  0.8345],
        [-1.3548,  0.8263],
        [-0.2921, -0.6292],
        [ 0.5261, -1.2520],
        [-1.2366,  1.0383],
        [-0.1763, -0.4746]])

In [38]:
ones @ x[0]

tensor([[ 0.1839, -0.0461],
        [-2.1231,  0.1071],
        [-2.7421,  0.9416],
        [-4.0969,  1.7679],
        [-4.3890,  1.1387],
        [-3.8629, -0.1133],
        [-5.0995,  0.9250],
        [-5.2759,  0.4504]])

In [39]:
# lower triangle matrix gives running sum of tokens!

In [40]:
# taking row-wise mean gives the running average of token
ones = ones/torch.sum(ones, 1, keepdim=True)

In [41]:
ones

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [42]:
ones @ x[0]

tensor([[ 0.1839, -0.0461],
        [-1.0616,  0.0536],
        [-0.9140,  0.3139],
        [-1.0242,  0.4420],
        [-0.8778,  0.2277],
        [-0.6438, -0.0189],
        [-0.7285,  0.1321],
        [-0.6595,  0.0563]])

In [45]:
# another version using softmax to get the running average of tokens
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow = wei @ x
xbow[0]

tensor([[ 0.1839, -0.0461],
        [-1.0616,  0.0536],
        [-0.9140,  0.3139],
        [-1.0242,  0.4420],
        [-0.8778,  0.2277],
        [-0.6438, -0.0189],
        [-0.7285,  0.1321],
        [-0.6595,  0.0563]])

In [50]:
# Self attention
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# implement single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1) # -> (B, T, T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v
out[0]

tensor([[-0.0116,  0.3514,  0.4815,  0.1246,  0.4435,  0.7315,  0.6309, -1.0015,
          0.4592, -0.1097, -0.8395, -0.1097,  0.6435,  0.5284, -0.2838, -0.3912],
        [-0.3066, -0.8024,  0.1620, -0.5296,  0.5885,  0.3834,  0.3314, -0.6656,
          0.8035, -0.1144,  0.1087,  0.0571,  0.2831, -0.6898, -0.3565,  0.0056],
        [-0.3922, -0.9258,  0.3465, -0.4881,  0.3832, -0.0345,  0.2314, -0.7179,
          0.5428,  0.1856,  0.0660,  0.3425, -0.0366, -0.1870, -0.5295,  0.1066],
        [-0.1240,  0.0973,  0.4636,  0.0434,  0.3841,  0.4899,  0.5227, -0.8830,
          0.3926, -0.0197, -0.6146,  0.0302,  0.4134,  0.4579, -0.3454, -0.2588],
        [-0.1404,  0.0788,  0.4966,  0.0794,  0.3397,  0.4890,  0.4762, -0.8463,
          0.3142,  0.0170, -0.5917,  0.0414,  0.3786,  0.5177, -0.3627, -0.2426],
        [-0.5229, -0.6839,  0.3790,  0.0156,  0.0950,  0.2815, -0.0912, -0.0421,
         -0.0946,  0.0888,  0.4014,  0.1032, -0.1556, -0.0015, -0.4179,  0.1658],
        [-0.4787,  0.2

In [51]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3644, 0.6356, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1956, 0.4044, 0.4000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.7698, 0.0391, 0.1487, 0.0425, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.7474, 0.0021, 0.1546, 0.0493, 0.0467, 0.0000, 0.0000, 0.0000],
         [0.0426, 0.1997, 0.0759, 0.2358, 0.3924, 0.0536, 0.0000, 0.0000],
         [0.0163, 0.0742, 0.0927, 0.5942, 0.0254, 0.0496, 0.1477, 0.0000],
         [0.0359, 0.0012, 0.0565, 0.0105, 0.0615, 0.7301, 0.0777, 0.0266]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5057, 0.4943, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1390, 0.2375, 0.6235, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1476, 0.0479, 0.0456, 0.7589, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0864, 0.0389, 0.7824, 0.0612, 0.0311, 0.0000, 0.0000, 0.0000],
         [0.0388, 0.056

In [54]:
v[0]

tensor([[-0.0116,  0.3514,  0.4815,  0.1246,  0.4435,  0.7315,  0.6309, -1.0015,
          0.4592, -0.1097, -0.8395, -0.1097,  0.6435,  0.5284, -0.2838, -0.3912],
        [-0.4757, -1.4637, -0.0211, -0.9046,  0.6716,  0.1838,  0.1597, -0.4730,
          1.0009, -0.1171,  0.6522,  0.1527,  0.0766, -1.3881, -0.3982,  0.2330],
        [-0.4940, -1.0064,  0.6523, -0.3667,  0.0622, -0.6298,  0.1085, -0.8268,
          0.1205,  0.6361, -0.0840,  0.7555, -0.4836,  0.6775, -0.7825,  0.2221],
        [-0.5430,  0.7927, -0.0746,  0.8800,  0.1699,  0.3133,  0.3455,  0.6900,
         -0.4220, -0.5952,  0.4393, -0.0865, -0.3057,  0.1110,  0.1173,  0.0049],
        [-0.5919, -1.3770,  0.8482,  0.0309, -0.2373,  0.5116, -0.6309, -0.0641,
         -0.6199,  0.6478,  0.5518,  0.2266, -0.2713,  0.3297, -0.7412,  0.3171],
        [-0.5530,  0.4339, -0.0379, -0.0158, -0.1806, -0.2453,  0.1485,  0.3778,
          0.3647, -0.8460, -0.1273, -0.9049,  0.3157,  0.8631, -0.0692, -0.1219],
        [-0.2193, -0.0

In [1]:
## full code with multi-head attention and layer norm 

import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'mps'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5059
step 300: train loss 2.4196, val loss 2.4336
step 400: train loss 2.3500, val loss 2.3564
step 500: train loss 2.2964, val loss 2.3127
step 600: train loss 2.2408, val loss 2.2498
step 700: train loss 2.2054, val loss 2.2189
step 800: train loss 2.1626, val loss 2.1854
step 900: train loss 2.1241, val loss 2.1505
step 1000: train loss 2.1031, val loss 2.1304
step 1100: train loss 2.0710, val loss 2.1200
step 1200: train loss 2.0385, val loss 2.0795
step 1300: train loss 2.0250, val loss 2.0642
step 1400: train loss 1.9912, val loss 2.0356
step 1500: train loss 1.9712, val loss 2.0314
step 1600: train loss 1.9618, val loss 2.0464
step 1700: train loss 1.9418, val loss 2.0135
step 1800: train loss 1.9094, val loss 1.9943
step 1900: train loss 1.9093, val loss 1.9878
step 2000: train loss 1.8851, val loss 1.9953
step 2100: train loss 1.